In [1]:
%run abalone.ipynb

In [21]:
def choice_exec(epoch_count=10, mb_size=10, report=1):
    load_choice_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [22]:
def load_choice_dataset():
    with open('Iris.csv') as csvfile:   #데이터셋을 가져온다
        csvreader = csv.reader(csvfile)
        next(csvreader, None)        #첫 행을 읽지 않고 건너뛴다.
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 5, 3      #출력데이터를 3열로 설정
    data = np.zeros([len(rows), input_cnt+output_cnt])   #꽃 개체들의 입출력 벡터 정보를 저장할 data 행렬을 만들 때 크기 지정에 이용
    
    #데이터가 원핫벡터표현이 안되있으므로 원핫벡터표현으로 만들어줌
    for n, row in enumerate(rows):
        if row[5] == 'Iris-setosa': data[n, 5] = 1
        if row[5] == 'Iris-versicolor': data[n, 6] = 1
        if row[5] == 'Iris-virginica': data[n, 7] = 1
        data[n, :4] = row[:4]

In [23]:
def forward_postproc(output, y):
    # output은 로짓값
    # 로짓값 -> 소프트맥스 교차 엔트로피를 -> loss 연산
    entropy = softmax_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy) 
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    #평균(loss) 역산(편미분)
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    #엔트로피 함수 역산(편미분)
    g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    #손실 기울기
    return G_output

In [24]:
def eval_accuracy(output, y):
    # np.argmax()는 각 로짓값을 담고 있는 최대값의 인덱스를 뽑아줌
    estimate = np.argmax(output, axis=1)
    answer = np.argmax(y, axis=1)
    # 둘 인덱스 간의 위치가 일치하면 True(1) 아니면 False(0)
    correct = np.equal(estimate, answer)
    
    return np.mean(correct)

In [25]:
def softmax(x):
    # 각 열에서 최대를 고르고(피쳐에 해당하는 최대)
    max_elem = np.max(x, axis=1)
     # 각 행을 전치시켜서 최댓값을 뺴주고 (행벡터 -> 열벡터)
    diff = (x.transpose() - max_elem).transpose()
    # 자연상수 처리 (열벡터)
    exp = np.exp(diff)
    # 자연상수 처리 한 것들을 합치고 (열벡터)
    sum_exp = np.sum(exp, axis=1)
    # 시그모이드 함수 수식 적용하고 전치 (열벡터 -> 행벡터)
    probs = (exp.transpose() / sum_exp).transpose()
    return probs

#실제로 호출되지 않지만 학습을 위한 구현
#소프트맥스 함수 자체보다 교차엔트로피 값을 중심으로 이루어지기 때문
def softmax_derv(x, y):
    mb_size, nom_size = x.shape
    derv = np.ndarray([mb_size, nom_size, nom_size])
    #3중 반복문을 통한 야코비안 행렬
    for n in range(mb_size):
        for i in range(nom_size):
            for j in range(nom_size):
                derv[n, i, j] = -y[n,i] * y[n,j]
            derv[n, i, i] += y[n,i]
    return derv

#log함수 폭주를 막기위해 epsilon 값을 10^(-10)값을 사용함
#labels는 원-핫 벡터
def softmax_cross_entropy_with_logits(labels, logits):
    probs = softmax(logits)
    return -np.sum(labels * np.log(probs+1.0e-10), axis=1)

def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels